### ДЗ 4.3
Вам нужно обкачать новости (или статьи) с сайтов газет или аналогичных онлайн-изданий и поместить это в базу.

Используем модуль PyQuery:

In [300]:
import requests
from fake_useragent import UserAgent
from pyquery import PyQuery as pq
import random
import time
import re

ua = UserAgent(verify_ssl=False).firefox # отключим проверку подлинности узла, чтоб не тупил
base_uri = "http://tmbtk.ru"

links = []  # список ссылок
dates = []  # список дат новостей
categories = [] # список категорий

for number in range(150):  # пройдём по 150 страницам
    news_uri_by_page = f"{base_uri}/news/page{number}"

    session = requests.session()
    req = session.get(news_uri_by_page, headers={'User-Agent': ua})
    text = req.text
    
    news_items = pq(text).find("div.news_item")
    # print(news_items)
    for item in news_items:
        links.append(pq(item).find("a").attr("href"))
    
    time.sleep(random.uniform(0.1, 0.4))

Сколько мы собрали статей?
У нас ещё может быть попадос в том, что мы два раза "считали" какую-нибудь ссылку. Например, если кто-то добавил ещё одну статью, пока шёл краулинг.

In [301]:
import pandas as pd

print(len(links))
links = pd.unique(links).tolist()
print(len(links))



1500
1489


Сейчас мы будем извлекать все данные.

In [302]:
aggregated_data = {
    "link": links,
    "news_id": [],
    "day": [],
    "month": [],
    "year": [],
    "text": [],
    "category": []
# А количество просмотров засунуть оказалось невозможно. Ниже указала, почему...
}

for link in links:
    time.sleep(random.uniform(0.1, 0.4))
    # Извлечём ID
    regex = r"^\/news\/.*?\/(.*)\/$"
    news_id = re.compile(regex).findall(link)[0]
    # print(news_id)
    
    response = session.get(f"{base_uri}{link}")

    # Извлечём, сколько раз прочитали статью... но нет, это невозможно.
    # Юзеру видно при просмотре, а при программном вебреквесте - нет.
    # regex = r"^.*<span>Прочитали (.*) раз.*<\/span>.*$"
    # was_read = re.compile(regex).findall(response.text)
    # print(was_read)
    # print(response.text)

    # Извлечём заголовок новости и дату
    for meta in pq(response.text).find("meta"):
        if pq(meta).attr("name") == "Description":
            description = pq(meta).attr("content")
        if pq(meta).attr("property") == "og:image":
            image_url = pq(meta).attr("content")  # оттуда легче всего регуляркой дёрнуть
            regex = r"^http:\/\/media\.tmbtk\.ru\/news\/(.*)\/.*$"
            date = re.compile(regex).findall(image_url)[0]
            year = date[0:4]
            month = date[4:6]
            day = date[6:8]
    # print(description)
    # print(day, month, year)
    
    # Извлечём категорию
    for a in pq(response.text).find("a"):
        if pq(a).attr('class') == "select":
            category = a.text
    # print(category)
    
    # Извлечём текст
    article_content = pq(response.text).find("div.article_content")
    article_text = pq(article_content).find("div.text")
    text_itself = pq(article_text).find("p").text()
    # print(text_itself)
    
    # Добавим в объект aggregated_data, что удалось собрать:
    aggregated_data['news_id'].append(news_id)
    aggregated_data['day'].append(day)
    aggregated_data['month'].append(month)
    aggregated_data['year'].append(year)
    aggregated_data['text'].append(text_itself)
    aggregated_data['category'].append(category)
    

Время создать БД и сбросить туда все данные.

К сожалению, тэгов у нас нет и ничего такого вообще нет. Получится только одна таблица.

In [303]:
import sqlite3

db_name = "hw4.3.db"
conn = sqlite3.connect(db_name)

cur = conn.cursor()
create_schema_file_name = 'hw4.3.db.sql'
with open(create_schema_file_name, encoding='utf-8') as f:
    query = f.read()
cur.executescript(query)

OperationalError: table aggregated_data already exists

In [304]:
df = pd.DataFrame(aggregated_data)
df

,link,news_id,day,month,year,text,category
0,/news/society/16060/,16060,14,02,2020,"Пакет поправок, серьезно упрощающих процедуру ...",Общество
1,/news/sport/16059/,16059,14,02,2020,Первую весеннюю игру сезона 2019/2020 РПЛ футб...,Спорт
2,/news/incidents/16058/,16058,14,02,2020,13 февраля в Тамбове в 14:35 на улице Гагарина...,Происшествия
3,/news/culture/16057/,16057,14,02,2020,Со следующей недели в Тамбове начнутся курсы п...,Культура
4,/news/society/16056/,16056,14,02,2020,13 февраля в Жердевке сотрудниками ДПС был ост...,Общество
...,...,...,...,...,...,...,...
1484,/news/society/14555/,14555,19,10,2019,Организаторы второго сезона управленческого ко...,Общество
1485,/news/society/14554/,14554,18,10,2019,Промышленный комплекс Тамбовской области демон...,Общество
1486,/news/incidents/14553/,14553,18,10,2019,17 октября в одиннадцать часов ночи на 34-м ки...,Происшествия
1487,/news/incidents/14552/,14552,18,10,2019,17 октября в дежурную часть полицию г. Мичурин...,Происшествия


In [306]:
df.to_sql(
    name="aggregated_data",
    con=conn,
    if_exists='append',
    index=False
)

Посчитаем, например, количество записей по месяцам.

Помним, что у нас только первые 1500 записей. Неудивительно, что раньше октября 2019 года к нам ничего не попало в статистику.

In [310]:
query = """
SELECT count(),month,year
FROM aggregated_data
GROUP BY month
"""
cur.execute(query)
pd.DataFrame(cur.fetchall())

,0,1,2
0,334,1,2020
1,206,2,2020
2,143,10,2019
3,374,11,2019
4,432,12,2019


Любопытно узнать, в какой категории больше новостей:

In [316]:
query = """
SELECT count() as cnt,category
FROM aggregated_data
GROUP BY category
ORDER BY cnt DESC
"""
cur.execute(query)
pd.DataFrame(cur.fetchall())

,0,1
0,881,Общество
1,378,Происшествия
2,84,Культура
3,53,Экономика
4,38,Спорт
5,36,Политика
6,19,Репортер


Происходит ли что-то хорошее в Тамбове?

In [317]:
query = """
SELECT count() as cnt
FROM aggregated_data
WHERE text LIKE '%радость%'
ORDER BY cnt DESC
"""
cur.execute(query)
pd.DataFrame(cur.fetchall())

,0
0,8


In [318]:
conn.close()